In [ ]:
import numpy as np
from sklearn.utils import shuffle
import tensorflow.keras as ke
from google.colab import drive
import tensorflow as tf
import gc
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
X = np.load("/content/drive/MyDrive/trail/dev_data.npy",allow_pickle=True,encoding="bytes")
Y = np.load("/content/drive/MyDrive/trial/dev_labels.npy",allow_pickle=True,encoding="bytes")

In [ ]:
maxL = 30

# Old Models

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

model = tf.keras.Sequential()

model.add(tf.keras.layers.Reshape((30, 40), input_shape=(30, 40)))
model.add(tf.keras.layers.LSTM(128, activation='sigmoid', return_sequences=True))
model.add(tf.keras.layers.LSTM(128, activation='sigmoid', return_sequences=True, go_backwards=True))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, activation='sigmoid', return_sequences=True)))
model.add(tf.keras.layers.GlobalMaxPooling1D())
model.add(tf.keras.layers.Dense(46, activation='softmax'))

model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'],
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.1))

model.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_3 (Reshape)         (None, 30, 40)            0         
                                                                 
 lstm_9 (LSTM)               (None, 30, 128)           86528     
                                                                 
 lstm_10 (LSTM)              (None, 30, 128)           131584    
                                                                 
 bidirectional_3 (Bidirectio  (None, 30, 256)          263168    
 nal)                                                            
                                                                 
 global_max_pooling1d_2 (Glo  (None, 256)              0         
 balMaxPooling1D)                                                
                                                                 
 dense_3 (Dense)             (None, 46)               

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

# Define the input shape
input_shape = (None, 40, 1)  # Variable length sequences of 40x1 dimensional vectors

# Define the input layer
inputs = layers.Input(shape=input_shape)

# Add the convolutional layers
x = layers.Conv2D(filters=32, kernel_size=(3, 3), padding='same')(inputs)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)
x = layers.MaxPooling2D(pool_size=(2, 2))(x)
x = layers.Dropout(0.5)(x) # added () to dropout layer
x = layers.Conv2D(filters=64, kernel_size=(3, 3), padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)
x = layers.MaxPooling2D(pool_size=(2, 2))(x)

# Reshape for the LSTM layers
x = layers.Reshape((-1, 64))(x)

# Add the recurrent layers with attention mechanism
hidden_size = 128
x = layers.Bidirectional(layers.LSTM(units=hidden_size, return_sequences=True))(x)
x = layers.Attention()([x, x])
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.5)(x)
x = layers.Bidirectional(layers.LSTM(units=hidden_size))(x)
x = layers.BatchNormalization()(x)

# Add the output layer
x = layers.Dense(units=128, activation='relu')(x)
outputs = layers.Dense(units=46, activation='softmax')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'],
              optimizer=tf.keras.optimizers.Adam(lr=0.1))
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, 40, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, None, 40, 32  320         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, None, 40, 32  128        ['conv2d[0][0]']                 
 alization)                     )                                                             

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

# Define the input shape
input_shape = (30, 40, 1)  # Variable length sequences of 40x1 dimensional vectors

# Define the input layer
inputs = layers.Input(shape=input_shape)

# Add the convolutional layers
x = layers.Conv2D(filters=40, kernel_size=(2, 2), padding='same')(inputs)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)
x = layers.MaxPooling2D(pool_size=(2, 2))(x)
x = layers.Dropout(0.5)(x) # added () to dropout layer
# Reshape for the LSTM layers
x = layers.Flatten()(x)

# Add the recurrent layers

# Add the output layer
x = layers.Dense(units=128, activation='relu')(x)
outputs = layers.Dense(units=46, activation='softmax')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'],
              optimizer=tf.keras.optimizers.Adam(lr=0.001))
model.summary()


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 30, 40, 1)]       0         
                                                                 
 conv2d_4 (Conv2D)           (None, 30, 40, 40)        200       
                                                                 
 batch_normalization_6 (Batc  (None, 30, 40, 40)       160       
 hNormalization)                                                 
                                                                 
 activation_4 (Activation)   (None, 30, 40, 40)        0         
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 15, 20, 40)       0         
 2D)                                                             
                                                                 
 dropout_5 (Dropout)         (None, 15, 20, 40)        0   

# New Model Bi LSTM

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

# Define the input shape
input_shape = (None, 40)  # Variable length sequences of 40-dimensional vectors

# Define the input layer
inputs = layers.Input(shape=input_shape)

# Add the convolutional layer
x = layers.Conv1D(filters=32, kernel_size=3, activation='relu')(inputs)
x = layers.MaxPooling1D(pool_size=2)(x)
x = layers.Dropout(0.2)(x)

# Add more convolutional layers
x = layers.Conv1D(filters=64, kernel_size=3, activation='relu')(x)
x = layers.MaxPooling1D(pool_size=2)(x)
x = layers.Dropout(0.2)(x)

x = layers.Conv1D(filters=128, kernel_size=3, activation='relu')(x)
x = layers.MaxPooling1D(pool_size=2)(x)
x = layers.Dropout(0.2)(x)

# Add the recurrent layers
x = layers.LSTM(units=256, return_sequences=True)(x)
x = layers.BatchNormalization()(x)
x = layers.LSTM(units=128)(x)
x = layers.BatchNormalization()(x)

# Add the output layer
x = layers.Dense(units=128, activation='relu')(x)
outputs = layers.Dense(units=46, activation='softmax')(x)
model = tf.keras.Model(inputs=inputs, outputs=outputs)
model.summary()

Model: "model_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_23 (InputLayer)       [(None, None, 40)]        0         
                                                                 
 conv1d_5 (Conv1D)           (None, None, 32)          3872      
                                                                 
 max_pooling1d_5 (MaxPooling  (None, None, 32)         0         
 1D)                                                             
                                                                 
 dropout_3 (Dropout)         (None, None, 32)          0         
                                                                 
 conv1d_6 (Conv1D)           (None, None, 64)          6208      
                                                                 
 max_pooling1d_6 (MaxPooling  (None, None, 64)         0         
 1D)                                                      

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])

# Data Generator

In [ ]:
#Split
x= Spl(X)
x,y = shuffle(x,y)
x_t = x[:-25000]
y_t = y[:-25000]

x_v = x[-25000:]
y_v = y[-25000:]

In [ ]:
class DataGenerator(Sequence):
    def __init__(self, x, y, batch_size):
        self.x = x
        self.y = y
        self.batch_size = batch_size
        self.indices = np.arange(len(x))

    def __len__(self):
        return int(np.ceil(len(self.x) / self.batch_size))

    def __getitem__(self, idx):
        batch_indices = self.indices[idx*self.batch_size:(idx+1)*self.batch_size]
        batch_x = [self.x[i] for i in batch_indices]
        batch_y = self.y[batch_indices]
        max_length = max(len(x) for x in batch_x)
        padded_x = np.zeros((len(batch_x), max_length, 40))
        for i, x in enumerate(batch_x):
            padded_x[i, :len(x), :] = x
        return padded_x, batch_y

# Create data generators for training and validation
#train_generator = DataGenerator(x_t, y_t, batch_size=1000)
#val_generator = DataGenerator(x_v, y_v, batch_size=1000)


NameError: ignored

# Training Stuff

In [ ]:
y=np.hstack(Y)
y = np.eye(46)[y]

TensorShape([None, 64])

In [ ]:
#To observe the Minimum and Maximum
maxxx=0
for i in x:
  if np.min(i)<maxxx:
    maxxx=np.min(i)
print(maxxx)

-0.9530866


In [ ]:
def Spl(X):
  x=[]
  for i in range(len(X)):
      for j in range(len(X[i][1])-1):
          t = X[i][0][X[i][1][j]:X[i][1][j+1]]
          x.append(t/145)
      t = np.array(X[i][0][X[i][1][j]:])
      x.append(t/145)
  x = np.array(x,dtype=object)
  return x

In [ ]:
#Not following this approach anymore
x=ke.preprocessing.sequence.pad_sequences(x, padding="pre",maxlen=maxL)
x = np.reshape(x,(len(x),maxL,40,1))
x.shape

(312274, 30, 40, 1)

In [ ]:
model.fit(x,y,epochs=20,validation_split=0.1,verbose=1,batch_size=3000)

## New Fit Strategy

In [ ]:
model.fit_generator(generator=train_generator,
                    validation_data=val_generator,
                    epochs=200)

Epoch 1/200


<ipython-input-107-66b7c9a4c613>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator=train_generator,


109/109 [==============================] - 72s 56ms/step - loss: 1.4199 - categorical_accuracy: 0.5731 - val_loss: 1.3650 - val_categorical_accuracy: 0.5994
Epoch 2/200
109/109 [==============================] - 4s 34ms/step - loss: 1.4148 - categorical_accuracy: 0.5730 - val_loss: 1.3659 - val_categorical_accuracy: 0.6020
Epoch 3/200
109/109 [==============================] - 4s 34ms/step - loss: 1.4069 - categorical_accuracy: 0.5751 - val_loss: 1.3668 - val_categorical_accuracy: 0.6015
Epoch 4/200
109/109 [==============================] - 4s 38ms/step - loss: 1.4037 - categorical_accuracy: 0.5776 - val_loss: 1.3821 - val_categorical_accuracy: 0.6028
Epoch 5/200
109/109 [==============================] - 4s 33ms/step - loss: 1.3986 - categorical_accuracy: 0.5776 - val_loss: 1.3480 - val_categorical_accuracy: 0.6048
Epoch 6/200
109/109 [==============================] - 4s 34ms/step - loss: 1.3942 - categorical_accuracy: 0.5800 - val_loss: 1.3589 - val_categorical_accuracy: 0.6014
Epo

KeyboardInterrupt: ignored

In [ ]:
del X,Y,x,y

In [ ]:
X = np.load("/content/drive/MyDrive/trial/test_data.npy",allow_pickle=True,encoding="bytes")

In [ ]:
x=[]
for i in range(len(X)):
    for j in range(len(X[i][1])-1):
        t = X[i][0][X[i][1][j]:X[i][1][j+1]]
        x.append(t)
    t = np.array(X[i][0][X[i][1][j]:])
    x.append(t)
x = np.array(x,dtype=object)

In [ ]:
x=ke.preprocessing.sequence.pad_sequences(x, padding="pre",maxlen=maxL)
x = np.reshape(x,(len(x),maxL,40,1))

In [ ]:
import pandas as pd
gc.enable


<function gc.enable()>

In [ ]:
Y = model.predict(x,verbose=1, batch_size=100)

5710/5710 [==============================] - 62s 11ms/step


In [ ]:
Y=np.argmax(Y,axis=1)
ID = np.arange(len(Y))
df = pd.DataFrame(data = np.array([ID,Y]).T,columns = ["index","label"],index = None)
df.to_csv("Prediction.csv",index=False)

In [ ]:
model.save("Model")